In [ ]:
!pip install llama-index-llms-gemini llama-index
!pip install llama-index-vector-stores-astra-db
!pip install llama-index-embeddings-gemini
!pip install llama-index 'google-generativeai>=0.3.0' matplotlib
#https://developers.llamaindex.ai/python/framework-api-reference/storage/vector_store/astra_db/




  Using cached pillow-10.4.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached banks-2.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.75.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.75.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.74

In [ ]:

from llama_index.embeddings.gemini import GeminiEmbedding

import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="this is a document"
)
embed_model.get_query_embedding("hello")


/var/folders/tz/8b88qlx557g0hfgcm5j26kr00000gn/T/ipykernel_14410/1567247874.py:10: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(


⚠️ Rate limit hit. Waiting 60 seconds... (Attempt 1/3)


In [ ]:
from google import genai
import os
import time
from llama_index.core.embeddings import BaseEmbedding
from typing import List

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize Google GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Create a custom embedding wrapper that inherits from BaseEmbedding
class CustomGeminiEmbedding(BaseEmbedding):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def _get_query_embedding(self, query: str) -> List[float]:
        """Get embedding for query text"""
        time.sleep(0.5)  # Small delay to avoid rate limits
        result = client.models.embed_content(
            model="gemini-embedding-001",
            contents=query
        )
        return result.embeddings[0].values
    
    def _get_text_embedding(self, text: str) -> List[float]:
        """Get embedding for document text"""
        time.sleep(0.5)  # Small delay to avoid rate limits
        result = client.models.embed_content(
            model="gemini-embedding-001",
            contents=text
        )
        return result.embeddings[0].values
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        """Async get embedding for query text"""
        return self._get_query_embedding(query)
    
    async def _aget_text_embedding(self, text: str) -> List[float]:
        """Async get embedding for document text"""
        return self._get_text_embedding(text)

# Create embedding model instance
embed_model = CustomGeminiEmbedding()

# Test it
test_embedding = embed_model.get_query_embedding("hello")
print(f"✅ Embedding dimension: {len(test_embedding)}")
print("Embedding model working successfully!")

✅ Embedding dimension: 3072
Embedding model working successfully!


In [ ]:
from llama_index.vector_stores.astra_db import AstraDBVectorStore
import os
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
)

ASTRA_DB_TOKEN = os.getenv("ASTRA_DB_TOKEN")

vector_db_store = AstraDBVectorStore(
    token=ASTRA_DB_TOKEN,
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    collection_name="course_info_gemini_3072",  
    embedding_dimension=3072,  
)

storage_context = StorageContext.from_defaults(vector_store=vector_db_store)

INDEX = VectorStoreIndex.from_vector_store(vector_db_store, embed_model=embed_model)


APICommander about to raise from: [{'message': "Collection already exists: trying to create Collection ('course_info') with different settings", 'errorCode': 'EXISTING_COLLECTION_DIFFERENT_SETTINGS', 'id': '74c2e92c-364d-4d05-84de-329c2a15d38b', 'family': 'REQUEST', 'title': 'Collection already exists', 'scope': 'EMPTY'}]
/var/folders/tz/8b88qlx557g0hfgcm5j26kr00000gn/T/ipykernel_32144/117577167.py:9: UserWarning: Collection 'course_info' is detected as having indexing turned on for all fields (either created manually or by older versions of this plugin). This implies stricter limitations on the amount of text each entry can store. Consider indexing anew on a fresh collection to be able to store longer texts.
  vector_db_store = AstraDBVectorStore(


In [19]:
from llama_index.core import Document
import time
def create_context(data, cat):
    contexts = []
    for index, docs in enumerate(data):
        contexts.append(
            Document(
                text=docs,
                metadata={"cat": cat, "time_stamp": time.time()},
            )
        )

    return contexts


def add_context(data, meeting_id):
    contexts = create_context(data, meeting_id)

    for context in contexts:
        INDEX.insert(context)


In [24]:
pishniaz = """
اخذ هر درس در مقطع کارشناسی مهندسی کامپیوتر به رعایت دو شرط کلیدی وابسته است که در جداول برنامه درسی مشخص شده‌اند: پيش نياز و هم نياز. این شرایط تضمین می‌کنند که دانشجو برای ورود به یک درس جدید، آمادگی علمی لازم را دارد.

### پيش نياز (Prerequisite)

**شرط پيش نياز به معنای آن است که شما باید درسی را قبلاً با موفقیت گذرانده باشید تا مجاز به ثبت‌نام در درس جدید باشید.**

به عبارت دیگر، درسی که به عنوان پيش نياز تعیین شده، پایه و اساس مفاهیم درس بعدی است و دانشجو باید نمره قبولی آن را کسب کرده باشد.

**مثال‌ها:**

*   برای اخذ درس **ریاضیات گسسته**، باید دروس رياضی عمومی 1 و مبانی کامپيوتر و برنامه نويسی را از پیش گذرانده باشید.
*   درس **معماری کامپيوتر** نیازمند گذراندن درس مدارهای منطقی به عنوان پيش نياز است.
*   درس **سیستم‌های عامل** را تنها پس از گذراندن معماری کامپیوتر می‌توانید اخذ کنید.
"""

hamniaz = """
### هم نياز (Corequisite)

**شرط هم نياز به این معناست که شما می‌توانید یک درس را به طور همزمان با درس مورد نظر اخذ کنید، یا اینکه آن را قبلاً با موفقیت گذرانده باشید.**

این شرط معمولاً برای دروس نظری و عملی (مانند آزمایشگاه‌ها) که لازم است همزمان با یکدیگر ارائه شوند تا مفاهیم نظری به صورت عملی پیاده‌سازی و تقویت شوند، به کار می‌رود.

**مثال‌ها:**

*   درس **مبانی کامپيوتر و برنامه نويسی** دارای کارگاه مبانی کامپيوتر و برنامه نويسی به عنوان هم نياز است. هدف این کارگاه، تقویت مهارت‌های برنامه نويسی با پیاده‌سازی عملی پروژه‌های مرتبط با مباحث نظری است.
*   درس **مدارهای منطقی** دارای آزمايشگاه مدارهای منطقی به عنوان هم نياز است.
*   درس **برنامه‌نويسی پيشرفته** دارای کارگاه برنامه‌نويسی پيشرفته به عنوان هم نياز است، که در آن فرصت پیاده‌سازی عملی مفاهیم آموخته شده در درس نظری فراهم می‌شود.

در نهایت، برای برنامه‌ریزی درسی، باید به جدول دروس و **برنامه نیمسالی پیشنهادی** توجه کنید که بر اساس همین روابط پيش نياز و هم نياز طراحی شده است تا مسیر تحصیلی شما به طور منطقی پیش برود.
"""

courses = """
در برنامه آموزشی مقطع کارشناسی مهندسی کامپیوتر، دانشجویان برای فارغ‌التحصیلی باید مجموعاً **۱۴۰ تا ۱۴۲ واحد** درسی را با موفقیت بگذرانند [۲۴]. این واحدها در پنج گروه اصلی دسته‌بندی می‌شوند که شامل واحدهای عمومی، پایه، اصلی، تخصصی و اختیاری است [۲۴], [۲۵].

### الف. دروس عمومی (۲۲ واحد)
گذراندن تمامی ۲۲ واحد عمومی برای کلیه دانشجویان الزامی است [۲۴]. این دروس شامل مباحثی چون علوم انسانی، معارف اسلامی، زبان و تربیت بدنی هستند [۳۳], [۳۴], [۳۵]. از این ۲۲ واحد، دانشجو باید دو درس را از گروه **مبانی نظری اسلام** (شامل انديشه اسالمی 9 و انديشه اسالمی 2 یا دروس مشابه) و یک درس را از هر یک از گروه‌های **اخالق در اسالم**، **انقالب اسالمی**، **تاریخ و تمدن اسالمی**، و **آشنایی با منابع اسالمی** انتخاب کند [۳۳], [۳۴]. سایر دروس عمومی اجباری عبارتند از زبان فارسی، زبان انگلیسی 1، زبان انگلیسی 2، تربیت بدنی 1، تربیت بدنی 2 و دانش خانواده و جمعیت [۳۴], [۳۵].

### ب. دروس پایه (۲۲ واحد)
گذراندن تمامی ۲۲ واحد دروس پایه برای دانشجویان مهندسی کامپیوتر الزامی است [۲۵]. این دروس، پایه‌های علمی لازم در حوزه‌های ریاضی و فیزیک را فراهم می‌کنند و شامل ریاضی عمومی ۱، ریاضی عمومی ۲، معادالت دیفرانسیل، آمار و احتمال مهندسی، فیزیک ۱، فیزیک ۲، آزمایشگاه فیزیک ۲ و کارگاه عمومی یا کارگاه برق هستند [۳۵], [۳۶].

### ج. دروس اصلی (۵۵ واحد)
تمامی ۵۵ واحد از دروس اصلی رشته مهندسی کامپیوتر، هسته تخصصی این برنامه را تشکیل می‌دهند و گذراندن آن‌ها اجباری است [۲۵]. این دروس دانش بنیادین کامپیوتر را پوشش می‌دهند و شامل: مبانی کامپیوتر و برنامه‌نویسی و کارگاه مبانی کامپیوتر و برنامه‌نویسی [۳۷], برنامه‌نویسی پیشرفته و کارگاه برنامه‌نویسی پیشرفته [۳۷], ریاضیات گسسته [۳۷], ساختمان داده‌ها و الگوریتم‌ها [۳۸], نظریه زبان‌ها و ماشین‌ها [۳۹], جبر خطی کاربردی [۳۹], مدارهای منطقی و آزمایشگاه مدارهای منطقی [۳۷], [۳۸], معماری کامپیوتر و آزمایشگاه معماری کامپیوتر [۳۹], ریزپردازنده و زبان اسمبلی و آزمایشگاه ریزپردازنده و زبان اسمبلی [۴۰], سیستم‌های عامل و آزمایشگاه سیستم‌های عامل [۴۰], شبکه‌های کامپیوتری و آزمایشگاه شبکه‌های کامپیوتری [۴۰], مدارهای الکتریکی و الکترونیکی و آزمایشگاه مدارهای الکتریکی و الکترونیکی [۳۸], [۳۹], مهندسی نرم‌افزار ۱ [۴۰], زبان تخصصی [۳۹]، و دروس پروژه‌محور مانند روش پژوهش و ارائه، کارآموزی و پروژه پایانی هستند [۴۱].

### د. دروس تخصصی (۲۴ واحد)
این بخش شامل ۲۴ واحد است که دانشجو باید آن‌ها را از میان درس‌هایی که در قالب **بسته‌های تخصصی** ارائه می‌شوند، انتخاب کند [۲۵]. مجموعاً چهار بسته تخصصی (هر بسته شامل ۸ درس سه واحدی است) موجود است [۲۵]. دانشجو باید از دو بسته تخصصی، یکی را به عنوان بسته اصلی (۱۸ واحد / ۶ درس) و دیگری را به عنوان بسته فرعی (۶ واحد / ۲ درس) انتخاب نماید [۲۵].

چهار بسته تخصصی ارائه شده عبارتند از:
1.  **بسته تخصصی طراحی و توسعه نرم‌افزار**: این بسته بر طراحی و توسعه یک سامانه نرم‌افزاری با کیفیت متمرکز است [۲۶]. دروس آن شامل طراحی الگوریتم‌ها، اصول طراحی پایگاه داده‌ها، اصول طراحی کامپایلر، زبان‌های برنامه‌نویسی، مهندسی نرم‌افزار ۲، آزمون نرم‌افزار، طراحی واسط کاربر و بازیابی اطلاعات است [۴۲], [۴۳].
2.  **بسته تخصصی سیستم‌های کامپیوتری**: این بسته مباحث مربوط به ساخت، طراحی و نگهداری اجزا سخت‌افزاری و نرم‌افزاری سامانه‌های کامپیوتری را پوشش می‌دهد [۲۸]. دروس این بسته شامل سیگنال‌ها و سیستم‌ها، طراحی مدارهای واسط، طراحی سیستم‌های دیجیتال برنامه‌پذیر، الکترونیک دیجیتال، سیستم‌های نهفته و بی‌درنگ، برنامه‌نویسی چندهسته‌ای، مبانی رایانش ابری و طراحی توأم سخت‌افزار و نرم‌افزار است [۴۴], [۴۵].
3.  **بسته تخصصی هوش مصنوعی**: هدف این بسته آشنایی با مفاهیم هوش مصنوعی، طراحی الگوریتم‌های هوشمند و کاربردهای آن نظیر رباتیک، داده‌کاوی و بیوانفورماتیک است [۲۹], [۳۰]. دروس اصلی آن عبارتند از طراحی الگوریتم‌ها، سیگنال‌ها و سیستم‌ها، مبانی و کاربردهای هوش مصنوعی، مبانی هوش محاسباتی، اصول علم ربات، مقدمه‌ای بر بیوانفورماتیک، داده‌کاوی و بازیابی اطلاعات [۴۵], [۴۶].
4.  **بسته تخصصی شبکه‌های کامپیوتری**: این بسته بر مفاهیم و مبانی شبکه‌های کامپیوتری و سیستم‌های توزیعی تمرکز دارد [۳۱]. دروس این بسته شامل سیگنال‌ها و سیستم‌ها، برنامه‌نویسی وب، انتقال داده‌ها، مبانی امنیت اطلاعات، سیستم‌های چندرسانه‌ای، برنامه‌نویسی دستگاه‌های سیار، مبانی رایانش ابری و مبانی اینترنت اشیاء است [۴۷], [۴۸].

### ه. دروس اختیاری (۱۵ واحد)
دانشجویان موظف به گذراندن ۱۵ واحد درسی اختیاری هستند که از فهرستی از دروس مصوب تعیین می‌شود [۲۵]. این دروس به دانشجو اجازه می‌دهد تا عمق دانش خود را در زمینه‌های خاص افزایش دهد [۳۲]. عناوینی چون گرافیک کامپیوتری، تعامل انسان و کامپیوتر، کارگاه ساخت ربات، طراحی بازی‌های کامپیوتری، تجارت الکترونیکی و سیستم‌های اتوماسیون صنعتی در این فهرست قرار دارند [۴۹], [۵۰].
"""


In [26]:
history = """
بر اساس کارنامه تحصیلی کیانا ناصری بادل بو، دروس گذرانده شده به همراه نمرات آن‌ها (از ۲۰) در طول پنج نیمسال اول به شرح زیر است: در **نیمسال اول**، دروس $\text{انديشه اسالمی 1}$ با نمره ۱۷/۵، $\text{رياضی عمومی 1}$ با نمره ۱۶، $\text{مبانی کامپيوتر و برنامه نويسی}$ با نمره ۱۷/۵، $\text{کارگاه مبانی کامپيوتر و برنامه نويسی}$ با نمره ۱۷/۵، و $\text{فيزيک 1}$ با نمره ۱۳/۷۵ با موفقیت گذرانده شدند [۱]. در این نیمسال در مجموع ۱۳ واحد درسی با میانگین ۱۶/۶۹ پاس شده است [۱]. در **نیمسال دوم**، دروس $\text{معارف اسالمی عملی}$ با نمره ۱۸، $\text{رياضی عمومی 2}$ با نمره ۱۷/۵، $\text{رياضيات گسسته}$ با نمره ۱۷، $\text{برنامه نويسی پيشرفته}$ با نمره ۲۰، $\text{کارگاه برنامه نويسی پيشرفته}$ با نمره ۲۰، $\text{فيزيک 2}$ با نمره ۱۵/۵، $\text{کارگاه برق}$ با نمره ۱۹/۵، و $\text{آزمايشگاه فيزيک}$ با نمره ۱۹/۵ گذرانده شدند [۱]. در این نیمسال ۱۸ واحد با میانگین ۱۸/۸۳ پاس شده است [۱]. در **نیمسال سوم**، دروس $\text{تربيت بدنی 1}$ با نمره ۲۰، $\text{انقالب اسالمی}$ با نمره ۲۰، $\text{زبان انگليسی 1}$ با نمره ۱۸/۵، $\text{معادالت ديفرانسيل}$ با نمره ۱۳/۵، $\text{آمار و احتمال مهندسی}$ با نمره ۱۹/۵، و $\text{معماری کامپيوتر}$ با نمره ۱۵/۷۵ با موفقیت گذرانده شده‌اند [۱]. در این نیمسال میانگین نمرات ۱۷/۸۹ برای ۱۵ واحد کسب شده است [۱]. در **نیمسال چهارم**، دروس $\text{تاريخ و تمدن اسالمی}$ با نمره ۱۹، $\text{آزمايشگاه مدارهای منطقی}$ با نمره ۱۹، $\text{ساختمان داده‌ها و الگوريتم‌ها}$ با نمره ۱۷/۷۵، $\text{نظريه زبان‌ها و ماشين‌ها}$ با نمره ۱۷/۲۵، $\text{طراحی الگوريتم‌ها}$ با نمره ۲۰، و $\text{اصول طراحی پايگاه داده‌ها}$ با نمره ۲۰ با موفقیت گذرانده شده‌اند [۱]. میانگین این نیمسال ۱۸/۰۵ برای ۱۶ واحد بوده است [۱]. در نهایت، در **نیمسال پنجم**، دروس $\text{تربيت بدنی 2}$ با نمره ۲۰، $\text{انقالب اسالمی}$ با نمره ۲۰، $\text{زبان اسمبلی}$ با نمره ۱۷، $\text{ريزپردازنده و زبان اسمبلی}$ با نمره ۲۰، $\text{سيستم‌های عامل}$ با نمره ۲۰، $\text{آزمايشگاه سيستم‌های عامل}$ با نمره ۲۰، $\text{شبکه‌های کامپيوتری}$ با نمره ۲۰، و $\text{مهندسی نرم‌افزار 1}$ با نمره ۱۹/۷۵ گذرانده شدند [۱]. میانگین نمرات نیمسال پنجم ۱۹/۱۲ برای ۱۹ واحد کسب شده است [۱].
"""

In [ ]:
#add_context([pishniaz], "pishniaz")

In [23]:
#add_context([hamniaz], "hamniaz")


In [25]:
#add_context([courses], "courses")


In [ ]:
#add_context([history], "history")


In [43]:

from llama_index.core.postprocessor import SimilarityPostprocessor


def retrieve_context(input_questions, top_k=3):
    """
    Retrieve relevant contexts from a database based on input questions and a meeting ID.

    Args:
        input_questions (list of str or str): A list of questions or a single question to query the database.
        meeting_id (str): The ID of the meeting to filter the context.
        top_k (int, optional): The number of top similar contexts to retrieve. Defaults to 10.

    Returns:
        list: A list of contexts retrieved from the database that match the input questions and meeting ID.
    """
    if isinstance(input_questions, str):
        input_questions = [input_questions]

    
    query_engine = INDEX.as_retriever(similarity_top_k=top_k)
    top_k = top_k // len(input_questions)
    contexts_of_db = []
    for input_question in input_questions:
        response = query_engine.retrieve(input_question)
        contexts_of_db.extend(response[:top_k])

    filtered_contexts =  filter_context(contexts_of_db)
    return [context.text for context in filtered_contexts]


def filter_context(contexts, similarity_threshold=0.80):
    postprocessor = SimilarityPostprocessor(similarity_cutoff=similarity_threshold)
    filtered_contexts = postprocessor.postprocess_nodes(contexts)

    return filtered_contexts



In [44]:
a = retrieve_context(['چه درس هایی پاس کرده؟'])
for i in a:
    print(i)

در این نیمسال میانگین نمرات ۱۷/۸۹ برای ۱۵ واحد کسب شده است [۱]. در **نیمسال چهارم**، دروس $	ext{تاريخ و تمدن اسالمی}$ با نمره ۱۹، $	ext{آزمايشگاه مدارهای منطقی}$ با نمره ۱۹، $	ext{ساختمان داده‌ها و الگوريتم‌ها}$ با نمره ۱۷/۷۵، $	ext{نظريه زبان‌ها و ماشين‌ها}$ با نمره ۱۷/۲۵، $	ext{طراحی الگوريتم‌ها}$ با نمره ۲۰، و $	ext{اصول طراحی پايگاه داده‌ها}$ با نمره ۲۰ با موفقیت گذرانده شده‌اند [۱]. میانگین این نیمسال ۱۸/۰۵ برای ۱۶ واحد بوده است [۱]. در نهایت، در **نیمسال پنجم**، دروس $	ext{تربيت بدنی 2}$ با نمره ۲۰، $	ext{انقالب اسالمی}$ با نمره ۲۰، $	ext{زبان اسمبلی}$ با نمره ۱۷، $	ext{ريزپردازنده و زبان اسمبلی}$ با نمره ۲۰، $	ext{سيستم‌های عامل}$ با نمره ۲۰، $	ext{آزمايشگاه سيستم‌های عامل}$ با نمره ۲۰، $	ext{شبکه‌های کامپيوتری}$ با نمره ۲۰، و $	ext{مهندسی نرم‌افزار 1}$ با نمره ۱۹/۷۵ گذرانده شدند [۱]. میانگین نمرات نیمسال پنجم ۱۹/۱۲ برای ۱۹ واحد کسب شده است [۱].
بر اساس کارنامه تحصیلی کیانا ناصری بادل بو، دروس گذرانده شده به همراه نمرات آن‌ها (از ۲۰) در طول پنج نیمسال اول به شرح زیر است: در **نی

In [40]:
len(a)

0